In [ ]:
from google.colab import files
files.upload()  # select your kaggle.json here

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"rohit678687687","key":"0de9b3de1a965e216df04267592b0980"}'}

In [ ]:
import os, shutil

# 1. Create the .kaggle folder
os.makedirs("/root/.kaggle", exist_ok=True)

# 2. Move uploaded kaggle.json into place
shutil.move("kaggle.json", "/root/.kaggle/kaggle.json")

# 3. Secure it
os.chmod("/root/.kaggle/kaggle.json", 0o600)


In [ ]:
!pip install -q kaggle

In [ ]:
# Download the Bird Audio Detection dataset
!kaggle datasets download -d langatfestus/bird-audio-detection -p /content/

# Create a folder and unzip into it
!mkdir -p /content/bird_audio_detection
!unzip -q /content/bird-audio-detection.zip -d /content/bird_audio_detection


Dataset URL: https://www.kaggle.com/datasets/langatfestus/bird-audio-detection
License(s): apache-2.0
^C
unzip:  cannot find or open /content/bird-audio-detection.zip, /content/bird-audio-detection.zip.zip or /content/bird-audio-detection.zip.ZIP.


In [ ]:
from google.colab import files
files.upload()  # ← Upload kaggle.json when it prompts you

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"rohit678687687","key":"0de9b3de1a965e216df04267592b0980"}'}

In [ ]:
import os
import shutil

# Create .kaggle directory and move kaggle.json
os.makedirs("/root/.kaggle", exist_ok=True)
shutil.move("kaggle.json", "/root/.kaggle/kaggle.json")

# Set file permission
os.chmod("/root/.kaggle/kaggle.json", 0o600)

In [ ]:
!pip install -q kaggle

In [ ]:
!kaggle datasets download -d soumendraprasad/sound-of-114-species-of-birds-till-2022 -p /content/
!unzip -q "/content/sound-of-114-species-of-birds-till-2022.zip" -d /content/birds_114

Dataset URL: https://www.kaggle.com/datasets/soumendraprasad/sound-of-114-species-of-birds-till-2022
License(s): CC0-1.0
100% 2.05G/2.06G [00:14<00:00, 75.3MB/s]
100% 2.06G/2.06G [00:14<00:00, 156MB/s] 


In [ ]:
!find /content/birds_114 -type f | head -n 10

/content/birds_114/Birds Voice.csv
/content/birds_114/Voice of Birds/Voice of Birds/Berlepschs Tinamou_sound/Berlepschs Tinamou5.mp3
/content/birds_114/Voice of Birds/Voice of Birds/Berlepschs Tinamou_sound/Berlepschs Tinamou11.mp3
/content/birds_114/Voice of Birds/Voice of Birds/Berlepschs Tinamou_sound/Berlepschs Tinamou2.mp3
/content/birds_114/Voice of Birds/Voice of Birds/Berlepschs Tinamou_sound/Berlepschs Tinamou8.mp3
/content/birds_114/Voice of Birds/Voice of Birds/Berlepschs Tinamou_sound/Berlepschs Tinamou12.mp3
/content/birds_114/Voice of Birds/Voice of Birds/Berlepschs Tinamou_sound/Berlepschs Tinamou6.mp3
/content/birds_114/Voice of Birds/Voice of Birds/Berlepschs Tinamou_sound/Berlepschs Tinamou13.mp3
/content/birds_114/Voice of Birds/Voice of Birds/Berlepschs Tinamou_sound/Berlepschs Tinamou4.mp3
/content/birds_114/Voice of Birds/Voice of Birds/Berlepschs Tinamou_sound/Berlepschs Tinamou7.mp3


In [ ]:
# 2️⃣– Convert ONLY valid .mp3 → .wav (skip broken files)
from pydub import AudioSegment
import os

src = "/content/birds_114"
dst = "/content/birds_114_wav"
os.makedirs(dst, exist_ok=True)

for root, _, files in os.walk(src):
    for f in files:
        if not f.lower().endswith(".mp3"):
            continue
        mp3 = os.path.join(root, f)
        wav = os.path.join(dst, os.path.splitext(f)[0] + ".wav")
        try:
            AudioSegment.from_file(mp3).export(wav, format="wav")
        except Exception as e:
            print(f" SKIPPING corrupt file: {f}")


In [ ]:
# Quick check
!find /content/birds_114_wav -type f -name "*.wav" | head -n 10

/content/birds_114_wav/Red-throated Piping Guan6.wav
/content/birds_114_wav/Berlepschs Tinamou8.wav
/content/birds_114_wav/Little Chachalaca6.wav
/content/birds_114_wav/Marail Guan16.wav
/content/birds_114_wav/Solitary Tinamou18.wav
/content/birds_114_wav/North Island Brown Kiwi6.wav
/content/birds_114_wav/Little Chachalaca19.wav
/content/birds_114_wav/Baudo Guan6.wav
/content/birds_114_wav/Darwins Nothura14.wav
/content/birds_114_wav/Grey-headed Chachalaca12.wav


In [ ]:
# 3️⃣– Prepare tf.data with SpecAugment
import tensorflow as tf
import os

DATA_DIR = "/content/birds_114_wav"

# Gather filepaths & labels
filepaths, labels = [], []
for fname in os.listdir(DATA_DIR):
    if not fname.endswith(".wav"): continue
    filepaths.append(os.path.join(DATA_DIR, fname))
    species = fname.split("_")[0]
    labels.append(species)

# Map species to indices
unique = sorted(set(labels))
label_index = {s:i for i,s in enumerate(unique)}
int_labels = [label_index[s] for s in labels]

# Create Dataset
ds = tf.data.Dataset.from_tensor_slices((filepaths, int_labels))
ds = ds.shuffle(len(filepaths), seed=42)
split = int(0.8 * len(filepaths))
ds_train = ds.take(split)
ds_val   = ds.skip(split)

AUTOTUNE = tf.data.AUTOTUNE
BATCH   = 32

def preprocess(path, label):
    audio = tf.io.read_file(path)
    wav, sr = tf.audio.decode_wav(audio, desired_channels=1)
    wav = tf.squeeze(wav, -1)
    # Pad/trim to 5s @16kHz
    target = 5 * 16000
    wav = wav[:target]
    wav = tf.pad(wav, [[0, target - tf.shape(wav)[0]]])
    # STFT → magnitude
    spec = tf.signal.stft(wav, frame_length=1024, frame_step=256)
    spec = tf.abs(spec)  # [time, bins]
    # Mel-scale
    mel = tf.tensordot(
        spec,
        tf.signal.linear_to_mel_weight_matrix(64, spec.shape[-1], sr, 80.0, 7600.0),
        1
    )
    mel = tf.math.log(mel + 1e-6)
    return tf.expand_dims(mel, -1), label

def spec_augment(mel, label):
    t = tf.shape(mel)[0]; f = tf.shape(mel)[1]
    # Time mask
    tm = tf.random.uniform([], 0, t//8, tf.int32)
    ts = tf.random.uniform([], 0, t - tm, tf.int32)
    mel = tf.concat([mel[:ts], tf.zeros([tm,f,1]), mel[ts+tm:]], 0)
    # Freq mask
    fm = tf.random.uniform([], 0, f//8, tf.int32)
    fs = tf.random.uniform([], 0, f - fm, tf.int32)
    mel = tf.concat([mel[:,:fs], tf.zeros([t,fm,1]), mel[:,fs+fm:]], 1)
    return mel, label

ds_train = ds_train.map(preprocess, num_parallel_calls=AUTOTUNE)
ds_train = ds_train.map(spec_augment,   num_parallel_calls=AUTOTUNE)
ds_train = ds_train.batch(BATCH).prefetch(AUTOTUNE)

ds_val   = ds_val.map(preprocess,   num_parallel_calls=AUTOTUNE)
ds_val   = ds_val.batch(BATCH).prefetch(AUTOTUNE)


In [ ]:
# 4️⃣– Build & compile a CNN with BatchNorm + Dropout
from tensorflow.keras import layers, models, callbacks
import tensorflow as tf

# Determine input shape
for x,_ in ds_train.take(1):
    in_shape = x.shape[1:]

# Calculate the flattened size after the last pooling layer
# Output shape of the last MaxPool2D is (None, 38, 8, 128)
flattened_size = 38 * 8 * 128

model = models.Sequential([
    layers.Input(shape=in_shape),

    layers.Conv2D(32,3,padding='same'), layers.BatchNormalization(), layers.Activation('relu'),
    layers.MaxPool2D(2),                  layers.Dropout(0.3),

    layers.Conv2D(64,3,padding='same'), layers.BatchNormalization(), layers.Activation('relu'),
    layers.MaxPool2D(2),                  layers.Dropout(0.3),

    layers.Conv2D(128,3,padding='same'),layers.BatchNormalization(), layers.Activation('relu'),
    layers.MaxPool2D(2),                  layers.Dropout(0.3),

    # Add a Reshape layer before Flatten
    layers.Reshape((flattened_size,)),
    layers.Flatten(), # This Flatten is now redundant but kept for clarity

    layers.Dense(256), layers.BatchNormalization(), layers.Activation('relu'),
    layers.Dropout(0.4),

    layers.Dense(len(unique), activation='softmax')
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 309, 64, 32)    │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 309, 64, 32)    │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_4 (Activation)       │ (None, 309, 64, 32)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 154, 32, 32)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 154, 32, 32)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 154, 32, 64)    │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 154, 32, 64)    │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_5 (Activation)       │ (None, 154, 32, 64)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 77, 16, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 77, 16, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 77, 16, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 77, 16, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_6 (Activation)       │ (None, 77, 16, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 38, 8, 128)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 38, 8, 128)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape (Reshape)               │ (None, 38912)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 38912)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 256)            │     9,961,728 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_7           │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_7 (Activation)       │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 2161)           │       555,377 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 10,611,697 (40.48 MB)

 Trainable params: 10,610,737 (40.48 MB)

 Non-trainable params: 960 (3.75 KB)

In [ ]:
# Grab one batch from train and val
for x_batch, y_batch in ds_train.take(1):
    print("Train batch X shape:", x_batch.shape)
    print("Train batch y shape:", y_batch.shape)
for x_batch, y_batch in ds_val.take(1):
    print("Val   batch X shape:", x_batch.shape)
    print("Val   batch y shape:", y_batch.shape)

# Check model input shape matches
print("Model expects input shape:", model.input_shape)


Train batch X shape: (32, 309, 64, 1)
Train batch y shape: (32,)
Val   batch X shape: (32, 309, 64, 1)
Val   batch y shape: (32,)
Model expects input shape: (None, 309, 64, 1)


In [ ]:
%%bash
# Clean and recreate the output directory
rm -rf /content/birds_114_wav_16bit
mkdir -p /content/birds_114_wav_16bit

# Convert each WAV to 16-bit PCM @16kHz, mono
for f in /content/birds_114_wav/*.wav; do
  name=$(basename "$f")
  ffmpeg -y -i "$f" -ac 1 -ar 16000 -sample_fmt s16 \
    "/content/birds_114_wav_16bit/$name" </dev/null &>/dev/null
done


In [ ]:
%%bash
# Confirm the directory now has .wav files
ls /content/birds_114_wav_16bit | head -n 5

# Check codec details of the first few files
ffprobe -hide_banner -show_streams /content/birds_114_wav_16bit/*.wav \
  | grep -E 'codec_name|bits_per_raw_sample' | head -n 4


Andean Guan10.wav
Andean Guan11.wav
Andean Guan12.wav
Andean Guan13.wav
Andean Guan14.wav


Argument '/content/birds_114_wav_16bit/Andean Guan11.wav' provided as input filename, but '/content/birds_114_wav_16bit/Andean Guan10.wav' was already specified.


In [ ]:
# ─── 1) Inspect the 16‑bit WAV folder ─────────────────────────────────────
import os

DATA_DIR = "/content/birds_114_wav_16bit"
all_files = [f for f in os.listdir(DATA_DIR) if f.endswith(".wav")]
print("Examples:", all_files[:5])
print("Total WAV files:", len(all_files))


Examples: ['Red-throated Piping Guan6.wav', 'Berlepschs Tinamou8.wav', 'Little Chachalaca6.wav', 'Marail Guan16.wav', 'Solitary Tinamou18.wav']
Total WAV files: 2161


In [ ]:
# ─── 2) Gather filepaths & labels ────────────────────────────────────────
filepaths = []
labels_str = []
for fname in all_files:
    filepaths.append(os.path.join(DATA_DIR, fname))
    # Extract species name as the part before the first underscore
    species = fname.split("_")[0]
    labels_str.append(species)

# Build a mapping from species name to integer index
unique_species = sorted(set(labels_str))
label_to_index = {name: idx for idx, name in enumerate(unique_species)}
print(f"Found {len(unique_species)} species")

# Convert string labels to integer labels
labels = [label_to_index[s] for s in labels_str]


Found 2161 species


In [ ]:
# ─── 3) Build train/val tf.data.Datasets ─────────────────────────────────
import tensorflow as tf

# Shuffle and split 80/20
dataset = tf.data.Dataset.from_tensor_slices((filepaths, labels))
dataset = dataset.shuffle(len(filepaths), seed=42)
split = int(0.8 * len(filepaths))
ds_train = dataset.take(split)
ds_val   = dataset.skip(split)

AUTOTUNE = tf.data.AUTOTUNE
BATCH    = 32


In [ ]:
# ─── 4) Preprocessing + SpecAugment functions ────────────────────────────
def preprocess(path, label):
    # 1. Read & decode WAV (mono)
    audio = tf.io.read_file(path)
    waveform, sample_rate = tf.audio.decode_wav(audio, desired_channels=1)
    waveform = tf.squeeze(waveform, axis=-1)  # shape: [samples]

    # 2. Pad or trim to exactly 5 seconds (5*16000=80000 samples)
    desired_len = 5 * 16000
    waveform = waveform[:desired_len]
    waveform = tf.pad(waveform, [[0, desired_len - tf.shape(waveform)[0]]])

    # 3. Compute STFT & magnitude spectrogram
    spec = tf.signal.stft(waveform,
                          frame_length=1024,
                          frame_step=256,
                          fft_length=1024)
    spec = tf.abs(spec)  # shape: [time, freq_bins]

    # 4. Convert to Mel-scale (64 bins)
    num_spectrogram_bins = spec.shape[-1]
    mel_weight_mat = tf.signal.linear_to_mel_weight_matrix(
        num_mel_bins=64,
        num_spectrogram_bins=num_spectrogram_bins,
        sample_rate=sample_rate,
        lower_edge_hertz=80.0,
        upper_edge_hertz=7600.0)
    mel_spec = tf.tensordot(spec, mel_weight_mat, 1)
    mel_spec.set_shape(spec.shape[:-1].concatenate([64]))

    # 5. Log scaling & add channel dimension
    log_mel_spec = tf.math.log(mel_spec + 1e-6)
    log_mel_spec = tf.expand_dims(log_mel_spec, -1)  # [time, mel_bins, 1]

    return log_mel_spec, label

def spec_augment(mel, label):
    # Time masking
    t = tf.shape(mel)[0]
    f = tf.shape(mel)[1]
    t_mask = tf.random.uniform([], 0, t//8, dtype=tf.int32)
    t_start = tf.random.uniform([], 0, t - t_mask, dtype=tf.int32)
    mel = tf.concat([mel[:t_start], tf.zeros([t_mask, f, 1]), mel[t_start+t_mask:]], axis=0)

    # Frequency masking
    f_mask = tf.random.uniform([], 0, f//8, dtype=tf.int32)
    f_start = tf.random.uniform([], 0, f - f_mask, dtype=tf.int32)
    mel = tf.concat([mel[:, :f_start], tf.zeros([t, f_mask,1]), mel[:, f_start+f_mask:]], axis=1)

    return mel, label


In [ ]:
# ─── 5) Assemble the final datasets ───────────────────────────────────────
ds_train = ds_train.map(preprocess, num_parallel_calls=AUTOTUNE)
ds_train = ds_train.map(spec_augment, num_parallel_calls=AUTOTUNE)
ds_train = ds_train.batch(BATCH).prefetch(AUTOTUNE)

ds_val = ds_val.map(preprocess, num_parallel_calls=AUTOTUNE)
ds_val = ds_val.batch(BATCH).prefetch(AUTOTUNE)


In [ ]:
# 6) Build the CNN model
from tensorflow.keras import layers, models

# Infer input shape from one batch
for x_batch, _ in ds_train.take(1):
    input_shape = x_batch.shape[1:]  # (time, mel_bins, 1)


model = models.Sequential([
    layers.Input(shape=input_shape),

    layers.Conv2D(32, 3, padding='same'),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.MaxPool2D(2),
    layers.Dropout(0.3),

    layers.Conv2D(64, 3, padding='same'),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.MaxPool2D(2),
    layers.Dropout(0.3),

    layers.Conv2D(128, 3, padding='same'),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.MaxPool2D(2),
    layers.Dropout(0.3),

    # Replace Flatten and the first Dense layer with Conv2D 1x1
    layers.Conv2D(256, 1, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.4),

    layers.Flatten(),

    layers.Dense(len(unique_species), activation='softmax')
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_9 (Conv2D)               │ (None, 309, 64, 32)    │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_12          │ (None, 309, 64, 32)    │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_12 (Activation)      │ (None, 309, 64, 32)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_9 (MaxPooling2D)  │ (None, 154, 32, 32)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_12 (Dropout)            │ (None, 154, 32, 32)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 154, 32, 64)    │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_13          │ (None, 154, 32, 64)    │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_13 (Activation)      │ (None, 154, 32, 64)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_10 (MaxPooling2D) │ (None, 77, 16, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_13 (Dropout)            │ (None, 77, 16, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 77, 16, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_14          │ (None, 77, 16, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_14 (Activation)      │ (None, 77, 16, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_11 (MaxPooling2D) │ (None, 38, 8, 128)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_14 (Dropout)            │ (None, 38, 8, 128)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_12 (Conv2D)              │ (None, 38, 8, 256)     │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_15          │ (None, 38, 8, 256)     │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_15 (Dropout)            │ (None, 38, 8, 256)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 77824)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 2161)           │   168,179,825 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 168,307,441 (642.04 MB)

 Trainable params: 168,306,481 (642.04 MB)

 Non-trainable params: 960 (3.75 KB)

In [ ]:
!pip install tensorflow
import tensorflow as tf
tf.config.optimizer.set_jit(False)   # ← disable XLA/JIT


In [ ]:
TIME_BINS = 1 + (5*16000 - 1024) // 256  # 309
MEL_BINS  = 64

def preprocess(path, label):
    audio = tf.io.read_file(path)
    waveform, sr = tf.audio.decode_wav(audio, desired_channels=1)
    waveform = tf.squeeze(waveform, -1)
    # pad/trim …
    waveform = waveform[:5*16000]
    waveform = tf.pad(waveform, [[0, 5*16000 - tf.shape(waveform)[0]]])
    # STFT
    spec = tf.abs(tf.signal.stft(waveform, 1024, 256))
    # Mel
    mel = tf.tensordot(
        spec,
        tf.signal.linear_to_mel_weight_matrix(
            MEL_BINS, spec.shape[-1], sr, 80.0, 7600.0
        ),
        1
    )
    # Log + static shape
    mel = tf.math.log(mel + 1e-6)
    mel = tf.ensure_shape(mel, [TIME_BINS, MEL_BINS])
    mel = tf.expand_dims(mel, -1)  # [309,64,1]
    return mel, label


In [ ]:
# Re‑import if needed
import tensorflow as tf

AUTOTUNE = tf.data.AUTOTUNE
BATCH    = 32

# Re‑shuffle & split
dataset = tf.data.Dataset.from_tensor_slices((filepaths, labels))
dataset = dataset.shuffle(len(filepaths), seed=42)
split   = int(0.8 * len(filepaths))
ds_train = dataset.take(split)
ds_val   = dataset.skip(split)

# Map preprocess & augment
ds_train = ds_train.map(preprocess, num_parallel_calls=AUTOTUNE)
ds_train = ds_train.map(spec_augment,   num_parallel_calls=AUTOTUNE)
ds_train = ds_train.batch(BATCH).prefetch(AUTOTUNE)

ds_val = ds_val.map(preprocess,   num_parallel_calls=AUTOTUNE)
ds_val = ds_val.batch(BATCH).prefetch(AUTOTUNE)


In [ ]:
from tensorflow.keras import layers, models

# Infer input shape
for x, _ in ds_train.take(1):
    input_shape = x.shape[1:]  # should be (309, 64, 1)

model = models.Sequential([
    layers.Input(shape=input_shape),

    layers.Conv2D(32, 3, padding='same'),
    layers.BatchNormalization(), layers.Activation('relu'),
    layers.MaxPool2D(2), layers.Dropout(0.3),

    layers.Conv2D(64, 3, padding='same'),
    layers.BatchNormalization(), layers.Activation('relu'),
    layers.MaxPool2D(2), layers.Dropout(0.3),

    layers.Conv2D(128, 3, padding='same'),
    layers.BatchNormalization(), layers.Activation('relu'),
    layers.MaxPool2D(2), layers.Dropout(0.3),

    layers.Flatten(),
    layers.Dense(256),
    layers.BatchNormalization(), layers.Activation('relu'),
    layers.Dropout(0.4),

    layers.Dense(len(unique_species), activation='softmax')
])

model.summary()

layers.Conv2D(128, 3, padding='same'),
layers.BatchNormalization(),
layers.Activation('relu'),
layers.MaxPool2D(2),
layers.Dropout(0.3),

# Instead of Flatten():
layers.GlobalAveragePooling2D(),

# Followed by your dense head:
layers.Dense(256),
layers.BatchNormalization(),
layers.Activation('relu'),
layers.Dropout(0.4),

layers.Dense(len(unique_species), activation='softmax')



Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_28 (Conv2D)              │ (None, 309, 64, 32)    │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_32          │ (None, 309, 64, 32)    │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_28 (Activation)      │ (None, 309, 64, 32)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_24 (MaxPooling2D) │ (None, 154, 32, 32)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_32 (Dropout)            │ (None, 154, 32, 32)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_29 (Conv2D)              │ (None, 154, 32, 64)    │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_33          │ (None, 154, 32, 64)    │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_29 (Activation)      │ (None, 154, 32, 64)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_25 (MaxPooling2D) │ (None, 77, 16, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_33 (Dropout)            │ (None, 77, 16, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_30 (Conv2D)              │ (None, 77, 16, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_34          │ (None, 77, 16, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_30 (Activation)      │ (None, 77, 16, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_26 (MaxPooling2D) │ (None, 38, 8, 128)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_34 (Dropout)            │ (None, 38, 8, 128)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_8 (Flatten)             │ (None, 38912)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 256)            │     9,961,728 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_35          │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_31 (Activation)      │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_35 (Dropout)            │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 2161)           │       555,377 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 10,611,697 (40.48 MB)

 Trainable params: 10,610,737 (40.48 MB)

 Non-trainable params: 960 (3.75 KB)

<Dense name=dense_15, built=False>

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

# Disable XLA if you still have issues:
import tensorflow as tf
tf.config.optimizer.set_jit(False)

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

callbacks = [
    EarlyStopping(monitor='val_loss',     patience=10, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6),
    ModelCheckpoint('best_model.h5',      monitor='val_accuracy', save_best_only=True, verbose=1)
]


In [ ]:
from tensorflow.keras import layers, models

# Infer input shape once
for x_batch, _ in ds_train.take(1):
    input_shape = x_batch.shape[1:]  # e.g. (309, 64, 1)

model = models.Sequential([
    layers.Input(shape=input_shape),

    # Block 1
    layers.Conv2D(32, 3, padding='same'),
    layers.BatchNormalization(), layers.Activation('relu'),
    layers.MaxPool2D(2),                   layers.Dropout(0.3),

    # Block 2
    layers.Conv2D(64, 3, padding='same'),
    layers.BatchNormalization(), layers.Activation('relu'),
    layers.MaxPool2D(2),                   layers.Dropout(0.3),

    # Block 3
    layers.Conv2D(128, 3, padding='same'),
    layers.BatchNormalization(), layers.Activation('relu'),
    layers.MaxPool2D(2),                   layers.Dropout(0.3),

    # <<< REPLACING Flatten with GAP >>>>
    layers.GlobalAveragePooling2D(),

    # Dense head
    layers.Dense(256),
    layers.BatchNormalization(), layers.Activation('relu'),
    layers.Dropout(0.4),

    layers.Dense(len(unique_species), activation='softmax')
])

model.summary()


Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_32 (Conv2D)              │ (None, 309, 64, 32)    │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_38          │ (None, 309, 64, 32)    │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_34 (Activation)      │ (None, 309, 64, 32)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_28 (MaxPooling2D) │ (None, 154, 32, 32)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_38 (Dropout)            │ (None, 154, 32, 32)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_33 (Conv2D)              │ (None, 154, 32, 64)    │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_39          │ (None, 154, 32, 64)    │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_35 (Activation)      │ (None, 154, 32, 64)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_29 (MaxPooling2D) │ (None, 77, 16, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_39 (Dropout)            │ (None, 77, 16, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_34 (Conv2D)              │ (None, 77, 16, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_40          │ (None, 77, 16, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_36 (Activation)      │ (None, 77, 16, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_30 (MaxPooling2D) │ (None, 38, 8, 128)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_40 (Dropout)            │ (None, 38, 8, 128)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 128)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 256)            │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_41          │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_37 (Activation)      │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_41 (Dropout)            │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 2161)           │       555,377 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 682,993 (2.61 MB)

 Trainable params: 682,033 (2.60 MB)

 Non-trainable params: 960 (3.75 KB)

In [ ]:
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

# Disable XLA to avoid JIT reshape issues
tf.config.optimizer.set_jit(False)

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

callbacks = [
    EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6),
    ModelCheckpoint('best_model.h5', monitor='val_accuracy', save_best_only=True, verbose=1)
]

history = model.fit(
    ds_train,
    validation_data=ds_val,
    epochs=100,
    callbacks=callbacks
)


Epoch 1/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 533ms/step - accuracy: 0.0021 - loss: 7.8598
Epoch 1: val_accuracy improved from -inf to 0.00231, saving model to best_model.h5


54/54 ━━━━━━━━━━━━━━━━━━━━ 49s 741ms/step - accuracy: 0.0020 - loss: 7.8625 - val_accuracy: 0.0023 - val_loss: 7.6206 - learning_rate: 0.0010
Epoch 2/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 472ms/step - accuracy: 4.6266e-04 - loss: 7.5657
Epoch 2: val_accuracy did not improve from 0.00231
54/54 ━━━━━━━━━━━━━━━━━━━━ 66s 572ms/step - accuracy: 4.6477e-04 - loss: 7.5668 - val_accuracy: 0.0000e+00 - val_loss: 7.4015 - learning_rate: 0.0010
Epoch 3/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 431ms/step - accuracy: 0.0046 - loss: 7.2755
Epoch 3: val_accuracy did not improve from 0.00231
54/54 ━━━━━━━━━━━━━━━━━━━━ 44s 628ms/step - accuracy: 0.0045 - loss: 7.2759 - val_accuracy: 0.0023 - val_loss: 7.2180 - learning_rate: 0.0010
Epoch 4/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 440ms/step - accuracy: 0.0016 - loss: 7.0131
Epoch 4: val_accuracy did not improve from 0.00231
54/54 ━━━━━━━━━━━━━━━━━━━━ 34s 636ms/step - accuracy: 0.0016 - loss: 7.0139 - val_accuracy: 0.0023 - val_loss: 6.9735 - learning_rate: 0.0010
Epoch 

54/54 ━━━━━━━━━━━━━━━━━━━━ 33s 618ms/step - accuracy: 0.0054 - loss: 6.5996 - val_accuracy: 0.0162 - val_loss: 6.6700 - learning_rate: 0.0010
Epoch 7/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 411ms/step - accuracy: 0.0079 - loss: 6.3394
Epoch 7: val_accuracy did not improve from 0.01617
54/54 ━━━━━━━━━━━━━━━━━━━━ 27s 511ms/step - accuracy: 0.0078 - loss: 6.3410 - val_accuracy: 0.0092 - val_loss: 6.4459 - learning_rate: 0.0010
Epoch 8/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 421ms/step - accuracy: 0.0154 - loss: 6.1995
Epoch 8: val_accuracy did not improve from 0.01617
54/54 ━━━━━━━━━━━━━━━━━━━━ 33s 617ms/step - accuracy: 0.0153 - loss: 6.2009 - val_accuracy: 0.0115 - val_loss: 6.1138 - learning_rate: 0.0010
Epoch 9/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 437ms/step - accuracy: 0.0205 - loss: 6.0276
Epoch 9: val_accuracy did not improve from 0.01617
54/54 ━━━━━━━━━━━━━━━━━━━━ 38s 568ms/step - accuracy: 0.0203 - loss: 6.0293 - val_accuracy: 0.0115 - val_loss: 5.9859 - learning_rate: 0.0010
Epoch 10/100
54/54

54/54 ━━━━━━━━━━━━━━━━━━━━ 29s 531ms/step - accuracy: 0.0152 - loss: 5.9237 - val_accuracy: 0.0462 - val_loss: 5.5592 - learning_rate: 0.0010
Epoch 11/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 436ms/step - accuracy: 0.0193 - loss: 5.8639
Epoch 11: val_accuracy did not improve from 0.04619
54/54 ━━━━━━━━━━━━━━━━━━━━ 29s 532ms/step - accuracy: 0.0192 - loss: 5.8640 - val_accuracy: 0.0254 - val_loss: 5.6401 - learning_rate: 0.0010
Epoch 12/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 429ms/step - accuracy: 0.0427 - loss: 5.5483
Epoch 12: val_accuracy improved from 0.04619 to 0.05774, saving model to best_model.h5


54/54 ━━━━━━━━━━━━━━━━━━━━ 34s 626ms/step - accuracy: 0.0425 - loss: 5.5497 - val_accuracy: 0.0577 - val_loss: 5.3004 - learning_rate: 0.0010
Epoch 13/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 428ms/step - accuracy: 0.0318 - loss: 5.5156
Epoch 13: val_accuracy improved from 0.05774 to 0.07390, saving model to best_model.h5


...

54/54 ━━━━━━━━━━━━━━━━━━━━ 33s 621ms/step - accuracy: 0.2375 - loss: 2.9228 - val_accuracy: 0.3649 - val_loss: 2.2921 - learning_rate: 0.0010
Epoch 40/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 421ms/step - accuracy: 0.2457 - loss: 2.8517
Epoch 40: val_accuracy did not improve from 0.36490
54/54 ━━━━━━━━━━━━━━━━━━━━ 36s 521ms/step - accuracy: 0.2451 - loss: 2.8544 - val_accuracy: 0.3649 - val_loss: 2.4189 - learning_rate: 0.0010
Epoch 41/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 433ms/step - accuracy: 0.2660 - loss: 2.7598
Epoch 41: val_accuracy did not improve from 0.36490
54/54 ━━━━━━━━━━━━━━━━━━━━ 47s 629ms/step - accuracy: 0.2655 - loss: 2.7614 - val_accuracy: 0.3118 - val_loss: 2.5684 - learning_rate: 0.0010
Epoch 42/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 394ms/step - accuracy: 0.2454 - loss: 2.7257
Epoch 42: val_accuracy did not improve from 0.36490
54/54 ━━━━━━━━━━━━━━━━━━━━ 39s 591ms/step - accuracy: 0.2450 - loss: 2.7279 - val_accuracy: 0.2540 - val_loss: 3.0408 - learning_rate: 0.0010
Epoch 43/100

54/54 ━━━━━━━━━━━━━━━━━━━━ 46s 614ms/step - accuracy: 0.2808 - loss: 2.5715 - val_accuracy: 0.3764 - val_loss: 2.0262 - learning_rate: 0.0010
Epoch 49/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 449ms/step - accuracy: 0.3064 - loss: 2.5277
Epoch 49: val_accuracy did not improve from 0.37644
54/54 ━━━━━━━━━━━━━━━━━━━━ 29s 546ms/step - accuracy: 0.3060 - loss: 2.5288 - val_accuracy: 0.3303 - val_loss: 2.2622 - learning_rate: 0.0010
Epoch 50/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 410ms/step - accuracy: 0.3219 - loss: 2.4067
Epoch 50: val_accuracy did not improve from 0.37644
54/54 ━━━━━━━━━━━━━━━━━━━━ 39s 518ms/step - accuracy: 0.3216 - loss: 2.4083 - val_accuracy: 0.3603 - val_loss: 2.2018 - learning_rate: 0.0010
Epoch 51/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 423ms/step - accuracy: 0.3591 - loss: 2.3071
Epoch 51: val_accuracy did not improve from 0.37644
54/54 ━━━━━━━━━━━━━━━━━━━━ 34s 619ms/step - accuracy: 0.3584 - loss: 2.3100 - val_accuracy: 0.3256 - val_loss: 2.3893 - learning_rate: 0.0010
Epoch 52/100

54/54 ━━━━━━━━━━━━━━━━━━━━ 32s 600ms/step - accuracy: 0.3150 - loss: 2.4114 - val_accuracy: 0.3903 - val_loss: 2.0428 - learning_rate: 0.0010
Epoch 53/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 393ms/step - accuracy: 0.3321 - loss: 2.2820
Epoch 53: val_accuracy improved from 0.39030 to 0.40647, saving model to best_model.h5


54/54 ━━━━━━━━━━━━━━━━━━━━ 32s 591ms/step - accuracy: 0.3315 - loss: 2.2840 - val_accuracy: 0.4065 - val_loss: 2.0118 - learning_rate: 0.0010
Epoch 54/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 428ms/step - accuracy: 0.3211 - loss: 2.3663
Epoch 54: val_accuracy did not improve from 0.40647
54/54 ━━━━━━━━━━━━━━━━━━━━ 37s 511ms/step - accuracy: 0.3209 - loss: 2.3684 - val_accuracy: 0.3279 - val_loss: 2.3794 - learning_rate: 0.0010
Epoch 55/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 426ms/step - accuracy: 0.3343 - loss: 2.3010
Epoch 55: val_accuracy did not improve from 0.40647
54/54 ━━━━━━━━━━━━━━━━━━━━ 47s 622ms/step - accuracy: 0.3337 - loss: 2.3037 - val_accuracy: 0.3118 - val_loss: 2.2956 - learning_rate: 0.0010
Epoch 56/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 427ms/step - accuracy: 0.3355 - loss: 2.2636
Epoch 56: val_accuracy improved from 0.40647 to 0.44111, saving model to best_model.h5


54/54 ━━━━━━━━━━━━━━━━━━━━ 34s 626ms/step - accuracy: 0.3353 - loss: 2.2649 - val_accuracy: 0.4411 - val_loss: 1.8810 - learning_rate: 0.0010
Epoch 57/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 426ms/step - accuracy: 0.3535 - loss: 2.1911
Epoch 57: val_accuracy did not improve from 0.44111
54/54 ━━━━━━━━━━━━━━━━━━━━ 34s 625ms/step - accuracy: 0.3529 - loss: 2.1929 - val_accuracy: 0.3995 - val_loss: 2.1139 - learning_rate: 0.0010
Epoch 58/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 424ms/step - accuracy: 0.3450 - loss: 2.2166
Epoch 58: val_accuracy did not improve from 0.44111
54/54 ━━━━━━━━━━━━━━━━━━━━ 33s 621ms/step - accuracy: 0.3445 - loss: 2.2189 - val_accuracy: 0.2933 - val_loss: 2.3257 - learning_rate: 0.0010
Epoch 59/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 431ms/step - accuracy: 0.3362 - loss: 2.3177
Epoch 59: val_accuracy did not improve from 0.44111
54/54 ━━━━━━━━━━━━━━━━━━━━ 29s 533ms/step - accuracy: 0.3358 - loss: 2.3182 - val_accuracy: 0.3256 - val_loss: 2.2593 - learning_rate: 0.0010
Epoch 60/100

In [ ]:
val_loss, val_acc = clf.evaluate(X_val, y_val, verbose=0)
print(f"Validation accuracy: {val_acc:.4f}")
